# Thermodynamic Feasibility and Sampling of Metabolite Concentrations
This notebook will demonstrate how **MASSpy** can be used to ensure thermodynamic feasibility in the metabolite concentrations of a model, and to generate samples of thermodynamically feasible metabolite concentrations for a model.

In [ ]:
# Disable gurobi logging output for this notebook.
try:
    import gurobipy
    gurobipy.setParam("OutputFlag", 0)
except ImportError:
    pass

import matplotlib.pyplot as plt

import numpy as np

import mass.test

from mass import MassConfiguration
from mass.thermo import ConcSolver

MASSCONFIGURATION = MassConfiguration()
# Load the JSON version of the textbook model
model = mass.test.create_test_model("textbook")

**Note**: Throughout this notebook, the term *thermodynamic feasibility constraint* for a reaction refers to the following:

For a given reaction:

$$
\textbf{S}^T \ln{(\textbf{x})} < \ln{(\text{Keq})}\ - \epsilon\ \text{if}\ \text{v}\ > 0\\
\textbf{S}^T \ln{(\textbf{x})} > \ln{(\text{Keq})}\ + \epsilon\ \text{if}\ \text{v}\ < 0\\
$$

where

* $\textbf{S}$ refers to the stoichiometry of the reaction
* $\textbf{x}$ refers to the vector of concentrations for the reaction metabolites
* $\text{Keq}$ refers to the equilibrium constant of the reaction
* $\text{v}$ refers to the reaction flux.
* $\epsilon$ refers to a buffer value for the constraint.

Based on methods outlined in <cite data-cite="KPH06">(Kummel et al., 2006)</cite> and <cite data-cite="HDR13">(Hamilton, et al., 2013)</cite>

## The ConcSolver Object

Upon initialization of the `ConcSolver` instance, the model becomes associated with the `ConcSolver` instance.
Metabolite concentrations and reaction equilibrium constants are added as variables to the `ConcSolver`, and thermodynamic feasibility constraints based on the reaction's flux direction and stoichiometry are created and added to the solver. Note that **all solver variables and constraints exist in logarithmic space.**

Metabolite concentrations that should be excluded from the solver can be defined using the `exclude_metabolites` argument (e.g. hydrogen and water). Reactions can also be excluded from the solver using the `exclude_reactions` argument.

Reactions that should exist at equilibrium or equilibrate very quickly should be set using the `equilibrium_reactions` argument. These reactions, such as the hemoglobin binding reactions and the adenylate kinase (ADK1) reaction, typically have a steady state flux value of 0.

In [ ]:
conc_solver = ConcSolver(
    model,
    excluded_metabolites=["h_c", "h2o_c"],
    excluded_reactions=None,
    equilibrium_reactions=["HBDPG", "HBO1", "HBO2", "HBO3", "HBO4", "ADK1",
                           "PFK_L"])
# View the model in the ConcSolver
conc_solver.model

The `ConcSolver` also becomes associated with the loaded model.

In [ ]:
print(model.conc_solver)

Concentrations and equilibrium constants cannot be negative numbers, therefore the bounds for each variable are set to ensure such behavior. Howevever, because $\ln(0)$ results in a domain error, the `ConcSolver` has the `zero_value_log_substitute` attribute. The value of the attribute is substituted for 0 to avoid any errors.

For example, if `zero_value_log_substitute=1e-10`, then taking the logarithm of 0 is treated as $\ln(0) \approx \ln(1*10^{-10}) = -23.026$

In [ ]:
print("Substitute for ln(0): ln({0:.1e})".format(
    conc_solver.zero_value_log_substitute))

Variables can be accessed through the `variables` attribute. The number of variables will equal the combined total of  the number of included metabolites and the number of included reactions. Specific variables can be accessed using their identifiers as a key.

In [ ]:
print("Number of included metabolites: {0}".format(len(conc_solver.included_metabolites)),
      "\nNumber of included reactions: {0}".format(len(conc_solver.included_reactions)),
      "\nTotal number of variables: {0}\n".format(len(conc_solver.variables)))

# Access the glucose concentration variable
variable = conc_solver.variables["glc__D_c"]
print("The glucose concentration variable",
      "\n----------------------------------\n",
      variable)

Constraints can be accessed through the `constraints` attribute. The number of constraints will equal the number of included reactions. Just like variables, specific constraints can be accessed using reaction identifiers as a key.

In [ ]:
print("Total number of constraints: {0}\n".format(len(conc_solver.constraints)))
# Access the hexokinase thermodynamic feasibility constraint
print("Thermodynamic feasibility constraint for HEX1",
      "\n-------------------------------------------\n",
      conc_solver.constraints["HEX1"])

Currently, the constraints do not have a buffer which provides some flexibility when solving the underlying mathematical problem of the `ConcSolver`. The `constraint_buffer` attribute can be used to set the *epsilon* value of the constraint. The constraints must be reset in order for the changed buffer value to take effect.

In [ ]:
conc_solver.constraint_buffer = 1e-7
conc_solver.reset_constraints()
print("Thermodynamic feasibility constraint for HEX1",
      "\n-------------------------------------------\n",
      conc_solver.constraints["HEX1"])

Upon initialization of the `ConcSolver`, the `ConcSolver.problem_type` is considered generic and no objective is set.

In [ ]:
print(conc_solver.problem_type)
print(conc_solver.objective)

The following sections will demonstrate different types of problems that can be solved using the `ConcSolver`.

## Solving for Feasible Concentrations
### Creating the QP problem

In order to determine thermodynamically feasible concentrations, a quadratic programming (QP) problem can be set up as follows:

Minimize
$$\ln( (\textbf{x}/\textbf{x}_0)^{2} )$$

subject to

$$
\textbf{S}^T \ln{(\textbf{x})} \lt \ln{(\text{Keq}_i)}\ - \epsilon\ \text{if}\ \text{v}_i\ \gt 0 \\
\textbf{S}^T \ln{(\textbf{x})} \gt \ln{(\text{Keq}_i)}\ + \epsilon\ \text{if}\ \text{v}_i\ \lt 0 \\
\ln(\text{Keq}_{i,\ lb}) \leq \ln(\text{Keq}_i) \leq \ln(\text{Keq}_{i,\ ub}) \\
\ln(\text{x}_{j,\ lb}) \leq \ln(\text{x}_j) \leq \ln(\text{x}_{j,\ ub}) \\
$$

where 

* $\textbf{S}$ refers to the stoichiometric matrix
* $\textbf{x}$ refers to the vector of metabolite concentrations
* $\textbf{x}_0$ refers to the vector of initial metabolite concentrations
* $\text{Keq}_i$ refers to the equilibrium constant of reaction $i$.
* $\text{v}_i$ refers to the flux for reaction $i$.
* $\text{x}_j$ refers to the concentration of metabolite $j$.
* $\epsilon$ refers to a buffer value for the constraint.

The first step is to set the optimization solver to one that is capable of handling quadratic objectives. Here, the [gurobi](https://www.gurobi.com/) solver will be used.

In [ ]:
conc_solver.solver = conc_solver.choose_solver(qp=True)
print(repr(conc_solver.solver))

To set up the underlying mathematical problem in the `ConcSolver`, the `setup_feasible_qp_problem` method can be used. The `fixed_conc_bounds` and `fixed_Keq_bounds` arguments can be used to set the upper and lower bounds of the corresponding variables equal to one other, fixing the variable's value. In this example, the metabolite concentrations will be allowed to change, while the equilibrium constants will be fixed at their original value.

In [ ]:
conc_solver.setup_feasible_qp_problem(
    fixed_Keq_bounds=conc_solver.model.reactions)

Using the `setup_feasible_qp_problem` method will also set the objective for the optimization.

In [ ]:
print(conc_solver.objective_direction)
conc_solver.objective

After using the `setup_feasible_qp_problem` method, the `ConcSolver` is ready for optimization and the `problem_type` is changed to reflect the current problem to be solved by the solver.

In [ ]:
print(conc_solver.problem_type)

### The ConcSolution Object
Once the `ConcSolver` is set up to solve the QP, the next step is to use the `optimize` method to solve the QP. A successful optimization will return a `ConcSolution` object. All values are transformed back into linear space upon being returned.

In [ ]:
conc_solution = conc_solver.optimize()
conc_solution

The `ConcSolution` object has several methods for viewing the results of the optimization and returning `pandas` objects containing the numerical solutions.

In [ ]:
dir(conc_solution)

In [ ]:
from mass.visualization import comparison_plot

If the visualization feature of **MASSPy** are enabled, the predicted values can be plotted against the original model values for comparison using the `comparison_plot` method.

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(6, 6))

# Compare values
comparison_plot(x=model, y=conc_solution, compare="concentrations",
                observable=conc_solver.included_metabolites, legend="right outside",
                xlabel="Current Concentrations [mM]", ylabel="Predicted Concentrations [mM]",
                plot_function="loglog", xy_line=True, xy_legend="best");

The model in the `ConcSolver` can be updated with the results contained within the `ConcSolution` using the `update_model_with_solution` method. Setting `inplace=True` will update the current model of the `ConcSolver`, while setting `inplace=False` will replace the model of the `ConcSolver` with a updated copy the model without modifying the original. Note that this also will remove the previous model's association with the `ConcSolver`.

In [ ]:
conc_solver.update_model_with_solution(conc_solution, concentrations=True, Keqs=False,
                                       inplace=False)
print("Same model object? {0}".format(conc_solver.model == model))
print(model.conc_solver)

## Concentration Sampling
### Basic usage

The easiest method of sampling concentrations is to use the `sample_concentrations` function in the `conc_sampling` submodule.

In [ ]:
from mass.thermo.conc_sampling import sample_concentrations

To set up the `ConcSolver` for sampling, the `setup_sampling_problem` method can be used. The `conc_percent_deviation` and `Keq_percent_deviation` arguments can be used to set the variable bounds for sampling. For this example, the defined concentrations will be allowed to deviate up to %75 from their baseline value, while the defined equilibrium constants will remain fixed at their current values. 

In [ ]:
conc_solver.setup_sampling_problem(
    conc_percent_deviation=0.75,
    Keq_percent_deviation=0)
print(conc_solver.problem_type)

To use the `sample_concentrations` function requires at least two arguments: a `ConcSolver` that has been set up for sampling, and the number of samples to generate.

In [ ]:
samples = sample_concentrations(conc_solver, n=20)
samples.head()

By default `sample_concentrations` uses the `optgp` method <cite data-cite="MHM14">
(Megchelenbrink, Huynen, and Marchiori, 2014)</cite> as it is suited for larger models and can run in parallel. By default the sampler uses the number of processes as defined in the `MassConfiguration` object. This can be changed by using the `processes` argument.

In [ ]:
print("One process:")
%time samples = sample_concentrations(conc_solver, n=1000, processes=1)
print("\nTwo processes:")
%time samples = sample_concentrations(conc_solver, n=1000, processes=2)

Alternatively the Artificial Centering Hit-and-Run for sampling <cite data-cite="KS98">
(Kaufman and Smith, 1998)</cite> can be utilized by setting the method to `achr`. The `achr` method does not support parallel execution but has good convergence and is almost Markovian.

In [ ]:
samples = sample_concentrations(conc_solver, n=100, method="achr")

In general, setting up the sampler is expensive since initial search directions are generated by solving many linear programming problems. Thus, it is recommended to generate as many samples as possible in one go. However, this might require finer control over the sampling procedure as described in the following section.

### Advance usage
#### Sampler objects

The concentration sampling process can be controlled on a lower level by using the sampler classes directly, found in the `conc_sampling` submodule.

In [ ]:
from mass.thermo.conc_sampling import ConcACHRSampler, ConcOptGPSampler

Both concentration sampler classes have standardized interfaces and take some additional arguments. 

For example, one such argument is the thinning factor, where “Thinning” means only recording samples every `x` iterations where `x` is the thinning factor. A higher thinning factors mean less correlated samples but also larger computation times.

By default the samplers use a thinning factor of 100 which creates roughly uncorrelated samples. Increasing the thinning factor leads to better mixing of samples, while lowering the thinning factor leads to more correlated samples. For example, to study convergence for a model, it may be desirable to set a thinning factor of 1 to obtain all iterates.

Samplers can also be seeded so that they produce the same results each time they are run.

In [ ]:
conc_achr = ConcACHRSampler(conc_solver, thinning=1, seed=5)
samples = conc_achr.sample(10, concs=True)
# Display only the first 5 samples
samples.head(5)

The sample function also comes with a `concs` argument that controls the sample output. Setting `concs=True` will return only concentration variables, while setting `concs=False` will return the equilibrium constant variables and any additional variables as well.

In [ ]:
samples = conc_achr.sample(10, concs=False)
print(samples.columns)

The `ConcOptGPSampler` has an additional `processes` argument specifying how many processes are used to create parallel sampling chains. This should be in the order of avaialble CPU cores for maximum efficiency. As noted before class initialization can take up to a few minutes due to generation of initial search directions. Sampling on the other hand is quick.

In [ ]:
conc_optgp = ConcOptGPSampler(conc_solver, processes=4, seed=5)

For the `ConcOptGPSampler`, the number of samples should be a multiple of the number of processes, otherwise it will be increased to the nearest multiple automatically.

In [ ]:
samples = conc_optgp.sample(10)
print("Number of samples generated: {0}".format(len(samples)))

#### Batch sampling

Sampler objects are made for generating billions of samples, however using the sample function might quickly fill up the computer RAM when working with genome-scale models. 

Here, the batch method of the sampler objects might come in handy. batch takes two arguments, the number of samples in each batch and the number of batches.

Suppose the concentration of ATP, ADP and AMP are not known. The batch sampler could be used to generate 10 batches of feasible concentrations with 100 samples each with, which could then be averaged to get the mean metabolite concentrations per batch. Finally, the mean metabolite concentrations and standard deviation could be calculated.

In [ ]:
# Remove current initial conditions for example
conc_solver.model.metabolites.atp_c.ic = None
conc_solver.model.metabolites.adp_c.ic = None
conc_solver.model.metabolites.amp_c.ic = None
# Set up concentration sampling problem
conc_solver.setup_sampling_problem(
    conc_percent_deviation=0.5,
    Keq_percent_deviation=0)

# Get batch samples
conc_optgp = ConcOptGPSampler(conc_solver, processes=4, seed=5)
batch_samples = [sample for sample in conc_optgp.batch(100, 10)]

# Determine average metabolite concentrations per batch, and per 
for met in ["atp_c", "adp_c", "amp_c"]:
    met = conc_solver.model.metabolites.get_by_id(met)
    per_batch_atp_ave = [
        np.mean(sample[met.id])
        for sample in batch_samples]
    print("Ave. {2} concentration: {0:.5f} +- {1:.5f}".format(
        np.mean(per_batch_atp_ave), np.std(per_batch_atp_ave), met.id))
    met.ic = np.mean(per_batch_atp_ave)